In [1]:
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Rectangle
import numpy as np


In [2]:
week_by_week2024 = nfl.import_weekly_data([2024], downcast=True)
play_by_play2024 = nfl.import_pbp_data([2024], downcast=True, cache=False, alt_path=None)
schedule2024 = nfl.import_schedules([2024])
seasonal2024 = nfl.import_seasonal_data([2024], "REG")
winstotal2024 = nfl.import_win_totals([2024])

Downcasting floats.
2024 done.
Downcasting floats.


In [3]:
# printing the columns of week_by_week so I don't mess up names lol
for columns in week_by_week2024.columns: 
    print (columns)

player_id
player_name
player_display_name
position
position_group
headshot_url
recent_team
season
week
season_type
opponent_team
completions
attempts
passing_yards
passing_tds
interceptions
sacks
sack_yards
sack_fumbles
sack_fumbles_lost
passing_air_yards
passing_yards_after_catch
passing_first_downs
passing_epa
passing_2pt_conversions
pacr
dakota
carries
rushing_yards
rushing_tds
rushing_fumbles
rushing_fumbles_lost
rushing_first_downs
rushing_epa
rushing_2pt_conversions
receptions
targets
receiving_yards
receiving_tds
receiving_fumbles
receiving_fumbles_lost
receiving_air_yards
receiving_yards_after_catch
receiving_first_downs
receiving_epa
receiving_2pt_conversions
racr
target_share
air_yards_share
wopr
special_teams_tds
fantasy_points
fantasy_points_ppr


In [4]:
# printing the columns of play_byPlay so I don't mess up names lol
for column in play_by_play2024.columns:
    print(column)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [5]:
for column in schedule2024.columns:
    print(column)

game_id
season
game_type
week
gameday
weekday
gametime
away_team
away_score
home_team
home_score
location
result
total
overtime
old_game_id
gsis
nfl_detail_id
pfr
pff
espn
ftn
away_rest
home_rest
away_moneyline
home_moneyline
spread_line
away_spread_odds
home_spread_odds
total_line
under_odds
over_odds
div_game
roof
surface
temp
wind
away_qb_id
home_qb_id
away_qb_name
home_qb_name
away_coach
home_coach
referee
stadium_id
stadium


In [28]:
# creating a function to calculate the win/loss ratio for each team given the seasonal data
def win_loss (schedule_df): 
    # making the dataframe smaller so its easier to handle the data
    smallSchedule2024 = schedule_df[['result', 'away_team', 'away_score', 'home_team', 'home_score', 'week']]   
    previousGames = smallSchedule2024[schedule_df['result'].notnull()]
    
    
    previousGames['home_win'] = np.where(previousGames['result'] > 0, 1, 0)
    previousGames['away_win'] = np.where(previousGames['result'] < 0, 1, 0)
    
    ## not sure if all this is necessary. I think the best way to do it would be to just count the 1s and 0s to figure out the record
    # obtaining the week list
    weekList = previousGames['week'].unique()
    
    #going week by week to calculate the records
    for week in weekList: 
        # getting the data for each week at a time (to tally up the win/loss at the specific time of playing)
        previousWeekData = previousGames[previousGames['week'] <= week]
        
        # calculating the wins and losses for each team
        homeWins = previousWeekData.groupby('home_team')['home_win'].sum()
        awayWins = previousWeekData.groupby('away_team')['away_win'].sum()
        homeLosses = previousWeekData.groupby('home_team')['home_win'].count() - previousWeekData.groupby('home_team')['home_win'].sum()
        awayLosses = previousWeekData.groupby('away_team')['away_win'].count() - previousWeekData.groupby('away_team')['away_win'].sum()
        
        teamWins = homeWins.add(awayWins, fill_value=0)
        teamLosses = homeLosses.add(awayLosses, fill_value=0)
        
        # creates a mask so we get the right week in the origianl df
        mask = schedule_df['week'] == week

        # adding the win/loss record to the schedule_df
        schedule_df.loc[mask, 'home_team_win_record'] = schedule_df['home_team'].map(teamWins)
        schedule_df.loc[mask, 'home_team_loss_record'] = schedule_df['home_team'].map(teamLosses)
        schedule_df.loc[mask, 'away_team_win_record'] = schedule_df['away_team'].map(teamWins)
        schedule_df.loc[mask, 'away_team_loss_record'] = schedule_df['away_team'].map(teamLosses)

    return schedule_df


scheduleWithRecords = win_loss(schedule2024) 
previousGames = scheduleWithRecords[scheduleWithRecords['result'].notnull()]
previousGames


C:\Users\ieyua\AppData\Local\Temp\ipykernel_4316\1382578560.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previousGames['home_win'] = np.where(previousGames['result'] > 0, 1, 0)
C:\Users\ieyua\AppData\Local\Temp\ipykernel_4316\1382578560.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previousGames['away_win'] = np.where(previousGames['result'] < 0, 1, 0)


,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,home_team_win_record,home_team_loss_record,away_team_win_record,away_team_loss_record
6706,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,20:20,BAL,20.0,KC,...,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,KAN00,GEHA Field at Arrowhead Stadium,1.0,0.0,0.0,1.0
6707,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,20:15,GB,29.0,PHI,...,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,SAO00,Arena Corinthians,1.0,0.0,0.0,1.0
6708,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,13:00,PIT,18.0,ATL,...,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,ATL97,Mercedes-Benz Stadium,0.0,1.0,1.0,0.0
6709,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,13:00,ARI,28.0,BUF,...,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,BUF00,New Era Field,1.0,0.0,0.0,1.0
6710,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,13:00,TEN,17.0,CHI,...,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,CHI98,Soldier Field,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6840,2024_09_DET_GB,2024,REG,9,2024-11-03,Sunday,16:25,DET,24.0,GB,...,Jordan Love,Dan Campbell,Matt LaFleur,Clete Blakeman,GNB00,Lambeau Field,6.0,3.0,7.0,1.0
6841,2024_09_LA_SEA,2024,REG,9,2024-11-03,Sunday,16:25,LA,26.0,SEA,...,Geno Smith,Sean McVay,Mike Macdonald,Clay Martin,SEA00,Lumen Field,4.0,5.0,4.0,4.0
6842,2024_09_IND_MIN,2024,REG,9,2024-11-03,Sunday,20:20,IND,13.0,MIN,...,Sam Darnold,Shane Steichen,Kevin O'Connell,Shawn Smith,MIN01,U.S. Bank Stadium,6.0,2.0,4.0,5.0
6843,2024_09_TB_KC,2024,REG,9,2024-11-04,Monday,20:15,TB,24.0,KC,...,Patrick Mahomes,Todd Bowles,Andy Reid,John Hussey,KAN00,GEHA Field at Arrowhead Stadium,8.0,0.0,4.0,5.0
